In [4]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv('/home/kcv/Desktop/HPPC_post_processing/post_processing/data/stitched_detail_data/RD_LFP_HPPC_REPT_Group0_150_0001_10_100.csv')


In [ ]:
df.columns

Index(['source_file', 'sheet_name', 'Record number', 'Sequence number of step',
       'Cycle No', 'Step No', 'Step name', 'Absolute time', 'Record time(m)',
       'Step time(h:m:s.ms)', 'volt(V)', 'Current(A)', 'Capacity(Ah)',
       'Energy(Wh)', 'Power(mW)', 'Internal R(Ω)', 'Charging energy(Wh)',
       'Discharge energy(Wh)', 'Charging capacity(Ah)',
       'Discharge capacity(Ah)', 'CUM charge E(Wh)', 'CUM discharge E(Wh)',
       'CUM charge C(Ah)', 'CUM discharge C(Ah)', 'Chamber T(℃)',
       'Chamber H(%)', 'Ripple F(Hz)', 'Ripple current(A)',
       'Total capacity(Ah)', 'DischargeVolt(V)', 'DischargeCurrent(A)',
       'DischargePower(mW)', 'water chiller outlet pressure(bar)',
       'water chiller return water pressure(bar)',
       'water chiller set flow(L/min)', 'water chiller set temperature(℃)',
       'Current water chiller temperature(℃)',
       'Current water chiller flow rate(L/min)', 'water chiller status',
       'water chiller mode', 'water chiller return wa

In [7]:
df['Cycle No'].unique()

array([1])

In [24]:
import pandas as pd

# ensure Absolute time is datetime
df['Absolute time'] = pd.to_datetime(df['Absolute time'])

# take absolute values BEFORE grouping
df['Current_abs'] = df['Current(A)'].abs()
df['Capacity_abs'] = df['Capacity(Ah)'].abs()

# sort for correct sequencing
df = df.sort_values(['Cycle No', 'Step No', 'Absolute time'])

# group and aggregate
durations = (
    df.groupby(['Cycle No', 'Step name', 'Step No'])
      .agg(
          Start=('Absolute time', 'min'),
          End=('Absolute time', 'max'),

          Current_min=('Current_abs', 'min'),
          Current_max=('Current_abs', 'max'),

          Capacity_min=('Capacity_abs', 'min'),
          Capacity_max=('Capacity_abs', 'max'),

          Volt_min=('volt(V)', 'min'),
          Volt_max=('volt(V)', 'max')
      )
)

# compute duration in seconds
durations['Duration'] = (durations['End'] - durations['Start']).dt.total_seconds()

# create c-rate (Current_max / 150), rounded to 2 decimals
durations['c_rate'] = (durations['Current_max'] / 150).round(1)

# drop start & end if not needed
durations = durations.drop(columns=['Start', 'End'])

durations


Current_min  Current_max  Capacity_min  \
Cycle No Step name Step No                                           
1        CCCV_Chg  2           2.499927    49.999626           0.0   
         CC_Chg    6          56.245018    56.249156           0.0   
                   11         56.245046    56.249206           0.0   
                   16         56.244939    56.249267           0.0   
                   21         56.245068    56.250058           0.0   
                   26         56.245069    56.249301           0.0   
                   31         56.244966    56.249971           0.0   
                   36         56.245058    56.249548           0.0   
                   41         56.244924    56.249318           0.0   
                   46         56.244916    56.249243           0.0   
         CC_DChg   4          74.995083    74.998978           0.0   
                   7          49.994989    49.998977           0.0   
                   9          74.995027    74.998984           0.0   
                   12         49.994914    49.995097           0.0   
                   14         74.994979    74.999021           0.0   
                   17         49.994925    49.995006           0.0   
                   19         74.994999    74.999033           0.0   
                   22         49.994978    49.995031           0.0   
                   24         74.995064    74.999286           0.0   
                   27         49.994929    49.994932           0.0   
                   29         74.995030    74.999074           0.0   
                   32         49.995034    49.995046           0.0   
                   34         74.994977    74.999032           0.0   
                   37         49.994902    49.994938           0.0   
                   39         74.994976    74.999916           0.0   
                   42         49.994935    49.999827           0.0   
                   44         74.995100    74.999037           0.0   
                   47         49.994964    49.999718           0.0   
         Rest      1           0.000000     0.000000           0.0   
                   3           0.000000     0.000000           0.0   
                   5           0.000000     0.000000           0.0   
                   8           0.000000     0.000000           0.0   
                   10          0.000000     0.000000           0.0   
                   13          0.000000     0.000000           0.0   
                   15          0.000000     0.000000           0.0   
                   18          0.000000     0.000000           0.0   
                   20          0.000000     0.000000           0.0   
                   23          0.000000     0.000000           0.0   
                   25          0.000000     0.000000           0.0   
                   28          0.000000     0.000000           0.0   
                   30          0.000000     0.000000           0.0   
                   33          0.000000     0.000000           0.0   
                   35          0.000000     0.000000           0.0   
                   38          0.000000     0.000000           0.0   
                   40          0.000000     0.000000           0.0   
                   43          0.000000     0.000000           0.0   
                   45          0.000000     0.000000           0.0   
                   48          0.000000     0.000000           0.0   

                            Capacity_max  Volt_min  Volt_max  Duration  c_rate  
Cycle No Step name Step No                                                      
1        CCCV_Chg  2               2.848  3.369122  3.650553   368.771     0.3  
         CC_Chg    6               0.157  3.419641  3.481131     9.995     0.4  
                   11              0.157  3.369363  3.414904     9.943     0.4  
                   16              0.156  3.359089  3.389535     9.980     0.4  
                   21              0.157 